Referal https://www.tensorflow.org/tutorials/audio/music_generation

In [4]:
from IPython import display
import collections
import datetime
import fluidsynth
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
import tensorflow as tf

from matplotlib import pyplot as plt
from typing import Dict, List, Optional, Sequence, Tuple

In [7]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Sampling rate for audio playback
_SAMPLING_RATE = 16000

# Download the maestro dataset of 1282 files
data_dir = pathlib.Path('data/maestro-v2.0.0')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'maestro-v2.0.0-midi.zip',
      origin='https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip',
      extract=True,
      cache_dir='.', cache_subdir='data',
  )

# these lines from tensor documentation above
filenames = glob.glob(str(data_dir/'**/*.mid*'))
print('Number of files:', len(filenames))
sample_file = filenames[1]
print(sample_file)

# data_dir = pathlib.Path('/Volumes/MAGIC1/CS50/myMusicGen/data/toto.mid')
# if not data_dir.exists():
#   tf.keras.utils.get_file(
#       'toto.mid',
#       origin='https://www.midiworld.com/download/5024',
#       extract=True,
#       cache_dir='.', cache_subdir='data',
#   )

# The data above contains 1200+ Midi files
# filenames = str(data_dir)

# Process a MIDI file
# sample_file = filenames
# print(sample_file)
# print(sample_file)

# generate a prettyMIDI object for the sample MIDI file
pm = pretty_midi.PrettyMIDI(sample_file)

Number of files: 1282
data/maestro-v2.0.0/2008/MIDI-Unprocessed_03_R1_2008_01-04_ORIG_MID--AUDIO_03_R1_2008_wav--1.midi


In [8]:
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=40):
    waveform = pm.fluidsynth(fs=float(_SAMPLING_RATE))
    # Take a sample of the generated waveform to mitigate kernel resets
    waveform_short = waveform[:seconds*_SAMPLING_RATE]
    return display.Audio(waveform_short, rate=_SAMPLING_RATE)

In [9]:
display_audio(pm)

In [10]:
print('Number of instruments:', len(pm.instruments))
for instrument in pm.instruments:
    # instrument = pm.instruments[instrument]
    instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
    print('Instrument name:', instrument_name)
    for i, note in enumerate(instrument.notes[:10]):
        note_name = pretty_midi.note_number_to_name(note.pitch)
        duration = note.end - note.start
        print(f'{i}: pitch={note.pitch}, note_name={note_name},'
                f' duration={duration:.4f}')

Number of instruments: 1
Instrument name: Acoustic Grand Piano
0: pitch=73, note_name=C#5, duration=0.3021
1: pitch=72, note_name=C5, duration=0.2174
2: pitch=70, note_name=A#4, duration=0.1680
3: pitch=69, note_name=A4, duration=0.1471
4: pitch=46, note_name=A#2, duration=0.7578
5: pitch=70, note_name=A#4, duration=0.3372
6: pitch=65, note_name=F4, duration=0.2852
7: pitch=66, note_name=F#4, duration=0.3099
8: pitch=63, note_name=D#4, duration=0.2826
9: pitch=58, note_name=A#3, duration=1.6302


In [11]:
for i, note in enumerate(instrument.notes[:10]):
  note_name = pretty_midi.note_number_to_name(note.pitch)
  duration = note.end - note.start
  print(f'{i}: pitch={note.pitch}, note_name={note_name},'
        f' duration={duration:.4f}')

0: pitch=73, note_name=C#5, duration=0.3021
1: pitch=72, note_name=C5, duration=0.2174
2: pitch=70, note_name=A#4, duration=0.1680
3: pitch=69, note_name=A4, duration=0.1471
4: pitch=46, note_name=A#2, duration=0.7578
5: pitch=70, note_name=A#4, duration=0.3372
6: pitch=65, note_name=F4, duration=0.2852
7: pitch=66, note_name=F#4, duration=0.3099
8: pitch=63, note_name=D#4, duration=0.2826
9: pitch=58, note_name=A#3, duration=1.6302


In [ ]:
def midi_to_notes(midi_file: str) -> pd.DataFrame:
  pm = pretty_midi.PrettyMIDI(midi_file)
  notes = collections.defaultdict(list)
  instrument_names = []
  for instrument in pm.instruments:
    instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
    instrument_names.append(instrument_name)
    # print(instrument_name)
    # Sort the notes by start time
    sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
    prev_start = sorted_notes[0].start
    # print(sorted_notes)

    for note in sorted_notes:
        start = note.start
        end = note.end
        notes['pitch'].append(note.pitch)
        notes['start'].append(start)
        notes['end'].append(end)
        notes['step'].append(start - prev_start)
        notes['duration'].append(end - start)
        prev_start = start
  
  return pd.DataFrame({name: np.array(value) for name, value in notes.items()}), instrument_names

In [ ]:
raw_notes, instrument_names_out = midi_to_notes(sample_file)
print(instrument_names_out)
raw_notes.head()


['Acoustic Grand Piano', 'Electric Guitar (clean)', 'Electric Bass (finger)', 'Ocarina', 'Electric Piano 2', 'String Ensemble 1', 'Lead 8 (bass + lead)', 'Choir Aahs', 'French Horn', 'Tuba', 'Pad 1 (new age)', 'Brass Section', 'Baritone Sax']


,pitch,start,end,step,duration
0,42,2.105264,2.165571,0.000000,0.060307
1,42,2.631580,2.691887,0.526316,0.060307
2,42,3.157896,3.218203,0.526316,0.060307
3,42,3.684212,3.744519,0.526316,0.060307
4,36,4.210528,4.270835,0.526316,0.060307


In [ ]:
get_note_names = np.vectorize(pretty_midi.note_number_to_name)
sample_note_names = get_note_names(raw_notes['pitch'])
sample_note_names[:10]

array(['F#2', 'F#2', 'F#2', 'F#2', 'C2', 'E2', 'E2', 'E2', 'E2', 'C2'],
      dtype='<U3')

In [ ]:
def notes_to_midi(notes_df: pd.DataFrame, out_file: str, instrument_name: List[str]) -> pretty_midi.PrettyMIDI:
    # Create a new PrettyMIDI object
    pm = pretty_midi.PrettyMIDI()
    print(instrument_name)
    # Create a dictionary to store the instruments
    instruments = {}

    # Add notes to the corresponding instruments
    for index, row in notes_df.iterrows():
        pitch = int(row['pitch'])
        start = float(row['start'])
        end = float(row['end'])
        duration = float(row['duration'])
        step = float(row['step'])

        # Find the instrument index based on the step value
        instrument_index = int(index % len(instrument_name))

        if instrument_index in instruments:
            # If the instrument already exists, use it
            instrument = instruments[instrument_index]
        else:
            # Create a new instrument
            if instrument_index == 9:
                # If it's a bass drum instrument, create a new drum instrument
                instrument = pretty_midi.Instrument(program=0, is_drum=True)
            elif instrument_index == 10:
                # If it's a snare drum instrument, create a new drum instrument
                instrument = pretty_midi.Instrument(program=0, is_drum=True)
            elif instrument_index == 11:
                # If it's a snare drum instrument, create a new drum instrument
                instrument = pretty_midi.Instrument(program=0, is_drum=True)
            else:
                # For non-drum instruments, create a new melody instrument
                program = pretty_midi.instrument_name_to_program(instrument_name[instrument_index])
                instrument = pretty_midi.Instrument(program=program)
            
            # Store the instrument in the dictionary
            instruments[instrument_index] = instrument

        # Create a new note
        note = pretty_midi.Note(
            velocity=100, pitch=pitch, start=start, end=end
        )

        # Add the note to the instrument
        instrument.notes.append(note)

    # Add the instruments to the MIDI object
    for instrument in instruments.values():
        pm.instruments.append(instrument)

    # Write the MIDI object to a file
    pm.write(out_file)

    # Return the created PrettyMIDI object
    return pm


In [ ]:
# Assuming you already have the `notes_df` DataFrame and `instrument_name` list

# Specify the output MIDI file path
example_file = 'example.midi'

# Call the `notes_to_midi` function
example_pm = notes_to_midi(raw_notes, out_file=example_file, instrument_name=instrument_names_out)

# Play the generated MIDI file in ipynb
display_audio(example_pm)


['Acoustic Grand Piano', 'Electric Guitar (clean)', 'Electric Bass (finger)', 'Ocarina', 'Electric Piano 2', 'String Ensemble 1', 'Lead 8 (bass + lead)', 'Choir Aahs', 'French Horn', 'Tuba', 'Pad 1 (new age)', 'Brass Section', 'Baritone Sax']
